# Go back to basics, figure out which class may have issue in predicting. Also try xgboost

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import scipy.stats as scs
import mskcc_functions as ski
import feature_engineering as fe
import xgboost as xgb

from matplotlib  import cm
from collections import Counter
from importlib import reload
from nltk import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import scale, normalize, robust_scale
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA

sns.set_context("paper")
%matplotlib inline

Import the data files

In [2]:
class_train = pd.read_csv('./data/training_variants')
text_train = pd.read_csv("./data/training_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])

<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<ipython-input-2-8825876aa8b2>:2: DeprecationWarning: invalid escape sequence \|
  text_train = pd.read_csv("./data/training_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])


<b>Function to create a table to look at true class vs predicted class</b>

In [3]:
# Create mock predicted labels
y = np.array(class_train['Class']).ravel()
yhat = np.random.randint(low=1, high=10, size=len(class_train)).ravel()

In [4]:
9 in yhat

True

In [5]:
def print_classification_table(y, yhat):
    '''
    INPUT:
    ======
    y : array
        array containing true class labels
    yhat: array
        array containing predicted class labels
    
    OUTPUT:
    =======
    table : DataFrame
        DataFrame containing classification results
    '''
    y_yhat = pd.DataFrame([y, yhat], index=['true_class', 'pred_class']).T
    table = y_yhat.groupby(['true_class','pred_class']).size().unstack('pred_class')
    table = table.fillna(value=0)
    num_classes = table.shape[0]
    table['class_count'] = 0
    table['truely_IDed'] = 0
    table['true_ID_rate'] = 0
    for i in range(num_classes):
        table.iloc[i, -3] = np.sum(table.iloc[i, :])
        table.iloc[i, -2]= table.iloc[i,i]
        table.iloc[i, -1] = (table.iloc[i, -2] / table.iloc[i, -3])
        
    return table

In [6]:
# test if the function workds
table = print_classification_table(y, yhat)
table

pred_class,1,2,3,4,5,6,7,8,9,class_count,truely_IDed,true_ID_rate
true_class,,,,,,,,,,,,
1,72,67,62,68,60,60,59,61,59,568.0,72,0.126761
2,39,60,55,47,48,49,54,53,47,452.0,60,0.132743
3,8,8,7,12,13,12,13,12,4,89.0,7,0.078652
4,70,76,77,70,75,82,91,83,62,686.0,70,0.102041
5,26,31,26,27,22,28,26,30,26,242.0,22,0.090909
6,33,33,28,33,23,38,34,30,23,275.0,38,0.138182
7,117,108,96,122,95,98,123,99,95,953.0,123,0.129066
8,1,3,2,2,3,3,1,3,1,19.0,3,0.157895
9,9,4,2,4,3,4,6,4,1,37.0,1,0.027027


<b>Prediction with Variants Info Only</b>
- Keep it simple by using data that appear in the variants file
- Predict with RandomForest, print the classification, figure out which part needs fiddling

In [7]:
# Import Data and extract gene and mutation type info
# Get Gene feature from 'train_variants' data
X_gene = np.array(class_train.Gene)
X_gene_int = LabelEncoder().fit_transform(X_gene.ravel()).reshape(-1, 1)
X_gene_bin = OneHotEncoder().fit_transform(X_gene_int).toarray()
gene_table = pd.DataFrame(X_gene_bin)

# Get Mutation Type from 'train_variants' data
mut_type = ski.convert_mutation_type(class_train)
X_mtype = np.array(mut_type['mutation_type'])
X_mtype_int = LabelEncoder().fit_transform(X_mtype.ravel()).reshape(-1, 1)
X_mtype_bin = OneHotEncoder().fit_transform(X_mtype_int).toarray()
mtype_table = pd.DataFrame(X_mtype_bin)

In [8]:
features1 = pd.concat([gene_table, mtype_table], axis=1)
X = np.array(features1).astype(float)
y = np.array(class_train.Class).astype(int).ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=55)

In [9]:
%%time
# Run RFC on the data
rfc = RandomForestClassifier(n_estimators=100, max_depth=100, n_jobs=4)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
y_proba = rfc.predict_proba(X_test)
print(accuracy_score(y_test, y_pred))
print(log_loss(y_test, y_proba))

0.575150300601
1.27738725603
CPU times: user 667 ms, sys: 9.36 ms, total: 677 ms
Wall time: 458 ms


In [10]:
# Assess with the table function abovev
print_classification_table(y_test, y_pred)

pred_class,1,2,4,5,6,7,8,9,class_count,truely_IDed,true_ID_rate
true_class,,,,,,,,,,,
1,46.0,2.0,24.0,8.0,8.0,7.0,0.0,0.0,95.0,46.0,0.484211
2,2.0,18.0,1.0,0.0,0.0,40.0,0.0,0.0,61.0,18.0,0.295082
3,0.0,0.0,2.0,2.0,0.0,9.0,0.0,0.0,13.0,2.0,0.153846
4,18.0,0.0,61.0,6.0,2.0,9.0,0.0,0.0,96.0,6.0,0.062500
5,1.0,0.0,3.0,14.0,4.0,16.0,0.0,0.0,38.0,4.0,0.105263
6,7.0,0.0,2.0,7.0,14.0,7.0,0.0,0.0,37.0,7.0,0.189189
7,1.0,20.0,0.0,0.0,3.0,130.0,0.0,0.0,154.0,0.0,0.000000
8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.000000
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,4.0,4.0,1.000000


In [23]:
accuracies = []
log_loss_scores = []

skf = StratifiedKFold(n_splits=10, shuffle=True)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    rfc = RandomForestClassifier(n_estimators=100, max_depth=100, n_jobs=4)
    rfc.fit(X_train, y_train)
    
    y_pred = rfc.predict(X_test)
    y_proba = rfc.predict_proba(X_test)
    
    accuracies.append(accuracy_score(y_test, y_pred))
    log_loss_scores.append(log_loss(y_test, y_proba, labels=list(range(1,10))))

print('Average Accuracy : %f' % np.mean(accuracies))
print('Average log loss : %f' % np.mean(log_loss_scores))

Average Accuracy : 0.589086
Average log loss : 1.269971


<b>Try XGBoost</b><br>
This code was adopted from the1owl's public kernel

In [12]:
# Adjust the class labels so it starts with 0
y_adj = y-1

# Split into test and train
x1, x2, y1, y2 = train_test_split(X, y_adj, test_size=0.15)

In [13]:
%%time
# Set up parameters for xgboost
param = params = {
        'eta': 0.03333,
        'max_depth': 4,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 9,
        'seed': 1,
        'silent': True
        }

watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]

model = xgb.train(params, xgb.DMatrix(x1, y1), 5000,  watchlist, verbose_eval=500, early_stopping_rounds=100)

[0]	train-mlogloss:2.16359	valid-mlogloss:2.16588
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[500]	train-mlogloss:1.14157	valid-mlogloss:1.22483
[1000]	train-mlogloss:1.05931	valid-mlogloss:1.16327
[1500]	train-mlogloss:1.01602	valid-mlogloss:1.13211
[2000]	train-mlogloss:0.991336	valid-mlogloss:1.11645
[2500]	train-mlogloss:0.976036	valid-mlogloss:1.10708
[3000]	train-mlogloss:0.96508	valid-mlogloss:1.10258
[3500]	train-mlogloss:0.957895	valid-mlogloss:1.09841
[4000]	train-mlogloss:0.952241	valid-mlogloss:1.09574
[4500]	train-mlogloss:0.947341	valid-mlogloss:1.09409
[4999]	train-mlogloss:0.942881	valid-mlogloss:1.09313
CPU times: user 32min 11s, sys: 746 ms, total: 32min 12s
Wall time: 4min 5s


In [14]:
score = log_loss(y2, model.predict(xgb.DMatrix(x2), ntree_limit=model.best_ntree_limit), labels = list(range(9)))
print(score)

1.0930910647


xgboost appears to do slightly better than RandomForest. I need to create a better features (better vector space for the text)

<b>Prediction with Variants info only - excluding point mutants and 'other'</b>

In [15]:
# get indices for rows that have 'Point Mutation' mutation type
pm_ind = list(np.argwhere(X_mtype == 'Point Mutation').ravel())
nonpm_ind = [ind for ind in range(len(X)) if ind not in pm_ind]

# Slice X and y
X_nonpm = X[nonpm_ind, :]
y_nonpm = y[nonpm_ind]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_nonpm, y_nonpm, test_size=0.15, random_state=55)

In [16]:
%%time
# Run RFC on the sliced dataset
rfc = RandomForestClassifier(n_estimators=100, max_depth=100, n_jobs=4)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
y_proba = rfc.predict_proba(X_test)
print(accuracy_score(y_test, y_pred))
print(log_loss(y_test, y_proba))

0.62


ValueError: y_true and y_pred contain different number of classes 5, 9. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [1 2 4 5 7]

I'd need to rewrite the function to account for this. The class that doesn't appear in the true labels was predicted to exist...

<b></b>

<b></b>

<b></b>

<b></b>

<b></b>

<b></b>

<b></b>